In [91]:
import os
import sys
import torch
sys.path.append(os.path.abspath("../common"))  # add path to common functions
from evaluate import evaluate_predictions
from preprocess import getdfs
from preprocess import one_hot_encode

In [92]:
train_df, test_df = getdfs("emoticon")
y_train = train_df["label"]
y_test = test_df["label"]

In [93]:
# Create one-hot encoding of the dataset

oh_X_train, oh_X_val, y_train, y_val = one_hot_encode(train_df, test_df)
X_tensor = torch.tensor(oh_X_train.values).float()
y_tensor = torch.tensor(y_train.values).float()


In [94]:
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [95]:
class OneHotNN(nn.Module):
    def __init__(self):
        super(OneHotNN, self).__init__()
        self.fc1 = nn.Linear(2159, 64)  # Input layer (features)  # cannot remove this
        self.fc2 = nn.Linear(64, 32)   # Hidden layer
        self.fc3 = nn.Linear(32, 1)    # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        x = torch.relu(self.fc2(x))  # Apply ReLU activation
        x = torch.sigmoid(self.fc3(x))             # Output layer (logits)
        return x
    
class OneHotNN2(nn.Module):
    def __init__(self):
        super(OneHotNN2, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2159, 512),
            nn.ReLU(),
            nn.Linear(512, 32),
            nn.ReLU(),
        )
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        x = self.linear_relu_stack(x)
        x = torch.sigmoid(self.fc(x))           # Output layer (logits)
        return x
model = OneHotNN2()


In [96]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate # HYPERPARAMETER


In [97]:
# Training loop
num_epochs = 20
batch_size = 64

for epoch in range(num_epochs):
    for i in range(0, len(X_tensor), batch_size):
        # Get the batch data
        batch_X = X_tensor[i:i + batch_size]
        batch_y = y_tensor[i:i + batch_size]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)  # Calculate loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/20], Loss: 0.4435
Epoch [2/20], Loss: 0.1639
Epoch [3/20], Loss: 0.0942
Epoch [4/20], Loss: 0.0903
Epoch [5/20], Loss: 0.0065
Epoch [6/20], Loss: 0.0143
Epoch [7/20], Loss: 0.0014
Epoch [8/20], Loss: 0.0007
Epoch [9/20], Loss: 0.0005
Epoch [10/20], Loss: 0.0003
Epoch [11/20], Loss: 0.0003
Epoch [12/20], Loss: 0.0002
Epoch [13/20], Loss: 0.0002
Epoch [14/20], Loss: 0.0001
Epoch [15/20], Loss: 0.0001
Epoch [16/20], Loss: 0.0001
Epoch [17/20], Loss: 0.0001
Epoch [18/20], Loss: 0.0001
Epoch [19/20], Loss: 0.0001
Epoch [20/20], Loss: 0.0001


In [98]:
# Evaluation
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    correct = 0
    total = len(X_tensor)

    outputs = model(X_tensor)
    for i in outputs:
        if i > 0.5:
            i = 1
        else:
            i = 0
    predicted = (outputs > 0.5).float()  # Apply thresholdts
    for i in range(len(outputs)):
        if (predicted[i] == y_tensor[i]):
            correct += 1

print(f"{correct*100/total:.2f}%")


100.00%


In [99]:
X_test_tensor = torch.tensor(oh_X_val.values).float()
y_test_tensor = torch.tensor(y_test.values).float()

model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    correct = 0
    total = len(X_test_tensor)

    outputs = model(X_test_tensor)
    for i in outputs:
        if i > 0.5:
            i = 1
        else:
            i = 0
    predicted = (outputs > 0.5).float()  # Apply thresholdts
    for i in range(len(outputs)):
        if (predicted[i] == y_test_tensor[i]):
            correct += 1

print(f"{correct*100/total:.2f}%")

91.00%


In [102]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)


1122369
